# MultinomialNaive Bayes using SKLearn Library

In [1]:
# Imports

import pandas as pd
import numpy as np
import nltk
import re

In [2]:
import requests

In [3]:
from sklearn.naive_bayes import MultinomialNB

In [4]:
cleaned_single_annotator = pd.read_csv("training_clean/clean_single_annotator.csv",delimiter=",")
cleaned_double_annotator_agree = pd.read_csv("training_clean/clean_double_annotator_agree.csv",delimiter=",")
cleaned_double_annotator_disagree = pd.read_csv("training_clean/clean_double_annotator_disagree.csv",delimiter=",")
cleaned_triple_annotator_agree = pd.read_csv("training_clean/clean_triple_annotator_agree.csv",delimiter=",")
cleaned_triple_annotator_disagree = pd.read_csv("training_clean/clean_triple_annotator_disagree.csv",delimiter=",")

In [5]:
cleaned_double_annotator_disagree.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat,freq
0,0,1001238,mengejar,2801,berencana pulang amerika serikat volume 42 ber...,1
1,1,1001238,mengejar,2802,berencana pulang amerika serikat volume 42 ber...,1
2,2,1001448,mengejar,2801,mekanisme memicu produksi nasional ketidakpuas...,1
3,3,1001448,mengejar,2802,mekanisme memicu produksi nasional ketidakpuas...,1
4,4,1001820,mengejar,2801,ganti kaum melayu melepaskan ketuanan melayuny...,1


In [6]:
# returns = requests.post('http://bahasa.cs.ui.ac.id/postag/API/tag',
#                          data={'Text[value]':'gambar dihasilkan layarnya cerah memiliki speaker menghasilkan suara keras jernih'})

## Tokenize sentences

In [7]:
cleaned_double_annotator_disagree['kalimat'] = cleaned_double_annotator_disagree['kalimat'].apply(nltk.word_tokenize)

In [8]:
cleaned_double_annotator_disagree.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat,freq
0,0,1001238,mengejar,2801,"[berencana, pulang, amerika, serikat, volume, ...",1
1,1,1001238,mengejar,2802,"[berencana, pulang, amerika, serikat, volume, ...",1
2,2,1001448,mengejar,2801,"[mekanisme, memicu, produksi, nasional, ketida...",1
3,3,1001448,mengejar,2802,"[mekanisme, memicu, produksi, nasional, ketida...",1
4,4,1001820,mengejar,2801,"[ganti, kaum, melayu, melepaskan, ketuanan, me...",1


In [10]:
cleaned_double_annotator_disagree.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat,freq
0,0,1001238,mengejar,2801,"[berencana, pulang, amerika, serikat, volume, ...",1
1,1,1001238,mengejar,2802,"[berencana, pulang, amerika, serikat, volume, ...",1
2,2,1001448,mengejar,2801,"[mekanisme, memicu, produksi, nasional, ketida...",1
3,3,1001448,mengejar,2802,"[mekanisme, memicu, produksi, nasional, ketida...",1
4,4,1001820,mengejar,2801,"[ganti, kaum, melayu, melepaskan, ketuanan, me...",1


## Trying to pick collocation range of 2

In [11]:
collocs = []
for row in cleaned_double_annotator_disagree.values:
    kata = row[2]
    kalimat = row[4]
    try:
        target_index = kalimat.index(kata)
    except ValueError:
        colloc = ['','','','']
        collocs.append(colloc)
        continue
    panjang_kalimat = len(kalimat)
    if target_index == 1:
        if panjang_kalimat - target_index > 2:
            colloc = ['',kalimat[target_index-1],kalimat[target_index+1],kalimat[target_index+2]]
        elif panjang_kalimat - target_index == 2:
            colloc = ['',kalimat[target_index-1],kalimat[target_index+1],'']
        else:
            colloc = ['',kalimat[target_index-1],'','']
    elif target_index == 0:
        if panjang_kalimat - target_index > 2:
            colloc = ['','',kalimat[target_index+1],kalimat[target_index+2]]
        elif panjang_kalimat - target_index == 2:
            colloc = ['','',kalimat[target_index+1],'']
    else:
        if panjang_kalimat - target_index > 2:
            colloc = [kalimat[target_index-2],kalimat[target_index-1],kalimat[target_index+1],kalimat[target_index+2]]
        elif panjang_kalimat - target_index == 2:
            colloc = [kalimat[target_index-2],kalimat[target_index-1],kalimat[target_index+1],'']
        else:
            colloc = [kalimat[target_index-2],kalimat[target_index-1],'','']
    collocs.append(colloc)
# collocs = np.array(collocs)

In [12]:
cleaned_double_annotator_disagree['colloc'] = collocs

In [14]:
cleaned_double_annotator_disagree.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat,freq,colloc
0,0,1001238,mengejar,2801,"[berencana, pulang, amerika, serikat, volume, ...",1,"[vermouth, berjanji, sherry, ]"
1,1,1001238,mengejar,2802,"[berencana, pulang, amerika, serikat, volume, ...",1,"[vermouth, berjanji, sherry, ]"
2,2,1001448,mengejar,2801,"[mekanisme, memicu, produksi, nasional, ketida...",1,"[muncul, film, kewajiban, mutunya]"
3,3,1001448,mengejar,2802,"[mekanisme, memicu, produksi, nasional, ketida...",1,"[muncul, film, kewajiban, mutunya]"
4,4,1001820,mengejar,2801,"[ganti, kaum, melayu, melepaskan, ketuanan, me...",1,"[melayu, dibantu, ketertinggalan, ekonominya]"


## Trying Classifier

In [15]:
clf = MultinomialNB()